In [ ]:
# Description:
# This notebook contains functions to perform correlation analysis between selected miRNAs and their predicted target genes. 
# It reads miRNA target predictions from three databases (DIANA, miRTarBase, TargetScan), filters mRNA expression 
# data based on differentially expressed (DE) genes and common target genes across these databases, 
# and then calculates Pearson correlations between miRNA and mRNA expression data.
#
# The functions:
# 1. Reads miRNA target predictions from DIANA, miRTarBase, and TargetScan.
# 2. Filters mRNA expression data to retain only DE genes that are common targets of the miRNAs.
# 3. Performs correlation analysis between the miRNA expression data and filtered mRNA expression data.
# 4. Outputs correlation coefficients and p-values for each miRNA-mRNA pair, saving results for those 
#    with significant correlations (p-value < 0.05 and negative correlation).

# Load necessary libraries
library(dplyr)
library(tidyr)
library(Hmisc)  
library(splitstackshape) 


In [ ]:
# Function to read gene lists from different databases
read_miRNA_data <- function(miRNA_id, diana_file, mirTarBase_file, targetScan_file) {
  diana_genes <- read.delim(diana_file)$gene_symbol
  mirTarBase_genes <- read.delim(mirTarBase_file, sep = ',')$Target
  targetScan_genes <- read.delim(targetScan_file)$Target.gene
  gene_list <- list('DIANA' = diana_genes, 'TargetScan' = targetScan_genes, 'miRTarBase' = mirTarBase_genes)
  return(gene_list)
}

In [ ]:
# Function to compile gene lists from databases and find common genes
get_common_genes <- function(gene_lists) {
  all_genes <- c(gene_lists$DIANA, gene_lists$TargetScan, gene_lists$miRTarBase)
  gene_counts <- table(all_genes)
  common_genes <- names(gene_counts[gene_counts >= 1])
  return(common_genes)
}

In [ ]:
# Function to filter DE genes and target genes
filter_expression_data <- function(mRNA_matrix, DE_genes, common_genes) {
  filtered_data <- mRNA_matrix %>%
    filter(Gene_ID %in% DE_genes$Gene_Symbol) %>%
    filter(Gene_ID %in% common_genes)
  rownames(filtered_data) <- filtered_data$Gene_ID
  filtered_data <- subset(filtered_data, select = -Gene_ID)
  return(filtered_data)
}

In [ ]:
# Function to process all steps for a specific miRNA
process_miRNA <- function(miRNA_id, diana_file, mirTarBase_file, targetScan_file, mRNA_matrix, DE_genes) {
  gene_lists <- read_miRNA_data(miRNA_id, diana_file, mirTarBase_file, targetScan_file)
  common_genes <- get_common_genes(gene_lists)
  filtered_data <- filter_expression_data(mRNA_matrix, DE_genes, common_genes)
  return(filtered_data)
}

In [ ]:
# Function to process miRNA expression, expand rows, and perform correlation with mRNA expression
perform_mirna_mrna_correlation <- function(miRNA_matrix, filtered_data_list, output_file_prefix) {
  
  # List of miRNAs and corresponding gene data
  miRNA_names <- c('hsa-miR-136-5p', 'hsa-miR-1-3p', 'hsa-miR-507', 'hsa-miR-514a-3p', 'hsa-miR-514a-5p', 'hsa-miR-513c-3p')
  
  # Iterate through each miRNA and corresponding gene data
  for (miRNA_name in miRNA_names) {
    # Select relevant miRNA expression and corresponding filtered gene expression data
    miRNA_expr <- miRNA_matrix[miRNA_name,]
    filtered_gene_expr <- filtered_data_list[[miRNA_name]]
    
    # Number of rows in filtered gene expression data
    num_rows_genes <- nrow(filtered_gene_expr)
    
    # Duplicate miRNA expression to match gene expression dimensions
    miRNA_expr_dup <- expandRows(miRNA_expr, count=num_rows_genes, count.is.col=FALSE)
    
    # Transpose for compatibility in correlation analysis
    miRNA_expr_dup <- t(as.matrix(miRNA_expr_dup))
    filtered_gene_expr <- t(as.matrix(filtered_gene_expr))
    
    # Perform Pearson correlation using rcorr
    correlation_result <- rcorr(miRNA_expr_dup, filtered_gene_expr, type='pearson')
    
    # Extract correlation coefficients and p-values
    corr_coefficients <- correlation_result$r[1, ]  # First row contains the correlation values
    p_values <- correlation_result$P[1, ]           # First row contains the p-values
    
    # Combine correlation coefficients and p-values into a single matrix
    combined_results <- cbind(corr_coefficients, p_values)
    
    # Filter results by correlation threshold (e.g., p-value < 0.05 and negative correlation)
    filtered_results <- combined_results[combined_results[, 2] < 0.05 & combined_results[, 1] < 0, ]
    
    # Define output file name based on miRNA name
    output_file <- paste0(output_file_prefix, '_', miRNA_name, '_correlation_results.csv')
    
    # Save filtered results to a CSV file
    write.table(filtered_results, file=output_file, sep='\t', col.names=NA)
    
    # Print the results for debugging purposes
    print(paste("Correlation results for", miRNA_name))
    print(filtered_results)
  }
}

In [ ]:

# Reading miRNA and mRNA expression data
miRNA_matrix <- read.delim('normalizedTMMlog2Cpmplus1_DE.csv')
mRNA_matrix <- read.delim('countDataFilt10CountsGeneSymbols.txt')
genes_DE_down <- read.delim("DGE_LIMMA_ovlp_DeSeq2_B_vs_C_DOWN.csv")

# Example: Process and save data for hsa-miR-136-5p
filtered_data_hsa_miR_136_5p_expr <- process_miRNA(
  miRNA_id = 'hsa-miR-136-5p',
  diana_file = 'hsa-miR-136-5p.tsv',
  mirTarBase_file = 'hsa-miR-136-5p.csv',
  targetScan_file = 'TargetScan8.0__miR-136-5p.predicted_targets.txt',
  mRNA_matrix = mRNA_matrix,
  DE_genes = genes_DE_down
)
save_filtered_data(filtered_data_hsa_miR_136_5p_expr, "filtered_data_hsa_miR_136_5p_expr.csv")

In [ ]:
# Example: Assuming other miRNAs like filtered_data_hsa_miR_507_expr are processed similarly
# Create a list of filtered data for correlation analysis
filtered_data_list <- list(
  'hsa-miR-136-5p' = filtered_data_hsa_miR_136_5p_expr,
  'hsa-miR-1-3p' = filtered_data_hsa_miR_1_3p_expr,
  'hsa-miR-507' = filtered_data_hsa_miR_507_expr,  # Add corresponding processing for miR-507
  'hsa-miR-514a-3p' = filtered_data_hsa_miR_514a_3p_expr,  # Add corresponding processing for miR-514a-3p
  'hsa-miR-514a-5p' = filtered_data_hsa_miR_514a_5p_expr,  # Add corresponding processing for miR-514a-5p
  'hsa-miR-513c-3p' = filtered_data_hsa_miR_513c_3p_expr   # Add corresponding processing for miR-513c-3p
)

# Perform correlation analysis and save the results
perform_mirna_mrna_correlation(miRNA_matrix, filtered_data_list, output_file_prefix='corr_coef')
